# Decision Trees 
 * It is a binary tree that recursively splits the datset until we are left with pure leaf nodes, that is the data with only one type of class. 
 *  Decision Trees are a non-parametric supervised learning method used for both classification and regression tasks. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features.
 * It is true that the tree is just a bunch of if else statement, But there exist many such trees to split the given data, so we have to find the optimal one.Thats why we this is considered as a machine learning algorithm.
 * Model will choose the split which maximises the information gain. 
 * Entropy is the measure of information contained in a state.If entropy is high, then we are very uncertain about the randomly picked point and we need more bits to descrive the statse.
 * Pure node is the node where entropy is minimum 
 
 To decide which attribute should be at the root node we can start by splitting the dataset wrt each feature and compute the gini impurity score. Consider the following splits based on three features.  
 <img src='https://miro.medium.com/max/980/1*S4byQQs8X2rNhJv4qm73uw.png'>
 <img src = 'https://miro.medium.com/max/980/1*j7WkOMGSLd17P_wlO_cRhw.png'> 

### Gini Impurity 
The Gini impurity is a measurement of the likelihood of incorrect classification of a new instance of a random variable, if that new instance were randomly classified according to the distribution of class labels from the data set. The lower the Gini score the better.
<img src = 'https://miro.medium.com/max/445/1*y78T1YV8wrb-DPbmNk2rKA.png'> 

<img src='https://miro.medium.com/max/980/1*3SMk52FU2a3TrDysEu7dkQ.png'> 

Suppose “Fever” has the lowest Gini impurity with a score of 0.386. In other words, “Fever” separates patients with and without flu the best.

<img src = 'https://miro.medium.com/max/850/1*uvngcDnw_vla-jWnFyyIJg.png'> 

We can apply this process to the right node as well. If the node itself has the lowest score then there is no point in separating the patients anymore and the node becomes the leaf node. This method is applied down the tree until we are only left with leaf nodes.

## Decision Tree with Numeric Values
What if the “Yes”/”No” answers in “Fever” were replaced with actual temperature readings ? How would we compile the tree then? Don’t panic, the process very similar to before. First we sort the values of the attribute in ascending order (smallest to largest). Then we compute the average for all adjacent patients. For example, if the first two values were 10 and 20, we’d get the average of those two values (average = (10+20)/2 = 15) and insert the value in between 10 and 20.
<img src = 'https://miro.medium.com/max/904/1*EWeBpqTq7_XhqvnSJwsMjg.png'>
Once we have inserted all the averages, we can use the averages to create a tree with a root node and two leaf nodes. In the picture above we have selected the value 37.5 (average of 37 and 38) and used it as the attribute in the root node. It asks the question: which patients have a fever less than 37.5 degrees celsius? If the answer is Yes (True) then it goes into the left node, otherwise it goes into the right node. Can you see that we follow the same notion as before. After fulfilling our node we must calculate the Gini impurity score for the node (with “Fever < 37.5” as the attribute). To obtain the Gini score we do the same as before: calculate Gini scores for the leaf nodes and then using weighted average methods we get the Gini impurtiy score for the root node.
This process is done for all averages. The average which returns the lowest Gini impurity score is selected to be the cut-off value in the root node or parent node.

## Notes: 
 * decision tree contains two nodes decision nodes and leaf nodes. Decision nodes contain condition and the condition is defined by feature index and the threshold value for that particular feature. 
 * The model tranverses through every feature and feature value for finding the best splitting condition which maximises the  information gain. 

## Entropy 
Entropy is a measure of information that indicates the disorder of the features with the target
<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR9d1OavwqcVz7tyymnWsnFHXRbOoKZunUumy4O3ziO_n7PReea87ixH400xaSC5RWzXQ&usqp=CAU"> 

## Information gain
The difference between entropy before and after splitting
<img src='https://qph.fs.quoracdn.net/main-qimg-dfad11c548327127fadd25ff992ace92'>

# Implimentation of decision tree from scratch 

In [2]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

In [4]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count) 
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

## Decision Tree Regressor 

The algorithm follows almost same process the difference lies in how we measure the information gain and how we make the prediction. Suppose our new data point reaches a particular leaf node. To make the prediction for regression problem we take the average value of y values in the node. 

To find the best split and threshold in decision tree regressor we use variance as a measure of impurity.(Just like entropy or gini index in the classification problem)

$VarReduction  = Var(parent) - \sum{w_i}* Var(child)$

#### Note : Decision Tree Split for numerical variables 
 * First we sort the feature then we go through all the unique feature values and compute the information gain.
 * decision tree takes a lot of time to train if we have numerical feature. 
 * time complexity will be increasing as the input is increasing. 
 * decision trees are the base of many ensemble tenchiques 
 * Decision Trees are robust to outliers because decision trees divide items by lines, so it does not difference how far is a point from lines.Most likely outliers will have a negligible effect because the nodes are determined based on the sample proportions in each split region (and not on their absolute values).

## Practise Questions 

 * How does decision tree works(entropy, inf gain, how to construct the tree, gini impurity) 
 * Understand the formula 
 * how to construct the tree with respect to categorical and numberical features 
 * What is the impact of outliers in decision tree 
 * Decision tree has low bias and high variance 
 * what is low bias and high variance. 
 * generalization error 
 * bias variance trade off 
 * full depth decision tree --> getting high variance 
 * how to convert high variance to low variance 
 * decision tree pruning 
 * How to perform pruning 
 * which all libraries you will use 
 * visualizaiton stuffs 
 * How decision tree regressor works 